In [1]:
from models.helper import GetModel
from dataloaders.train_loader import GSVCitiesDataset
from dataloaders.test_loader import TestDataset
from torch.utils.data import DataLoader
import torch
import os
from pytorch_metric_learning import losses, miners
from train import train_model
from evaluation import eval_model
from utils.lr_scheduler import custom_scheduler
import random
import numpy as np
import torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)


train_dataset = GSVCitiesDataset(generated_data_prob=0)
test_dataset = TestDataset()

num_workers = 16
train_dataloader = DataLoader(train_dataset, 
                              batch_size=100, 
                              shuffle=True, 
                              num_workers=num_workers, 
                              pin_memory=True
                              )

test_dataloader = DataLoader(test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
sf_xs_test_dataset = TestDataset(path='./data/sf_xs/test')
sf_xs_test_dataloader = DataLoader(sf_xs_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
tokyo_test_dataset = TestDataset(path='./data/tokyo_xs/test')
tokyo_test_dataloader = DataLoader(tokyo_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )




# Phase 1

### Experiment 1: Avg Pooling + Contrastive Loss + No Miner + SGD(lr=0.01, decay=1e-3)

In [5]:
TRAIN = False

EXPERIMENT_NAME = 'contrastive_avg_model'

model = GetModel(aggregator='average')
model = model.to('cuda')
if TRAIN:   
    miner = None
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2542728/111993283.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 69.76 | 80.93  |
+----------+-------+--------+
R1: 0.6976104091079695
R5: 0.8093331665207056


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 32.70 | 50.80  |
+----------+-------+--------+
R1: 0.327
R5: 0.508


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 45.08 | 61.90  |
+----------+-------+--------+
R1: 0.4507936507936508
R5: 0.6190476190476191


### Experiment 2: Gem Pooling + Contrastive Loss + No Miner + SGD(lr=0.01, decay=1e-3)

In [6]:
TRAIN = False

EXPERIMENT_NAME = 'contrastive_gem_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = None
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2542728/3850795405.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 70.81 | 81.33  |
+----------+-------+--------+
R1: 0.7081196046540723
R5: 0.8133366695858877


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 32.20 | 48.40  |
+----------+-------+--------+
R1: 0.322
R5: 0.484


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 42.86 | 62.22  |
+----------+-------+--------+
R1: 0.42857142857142855
R5: 0.6222222222222222


# Phase 2

#### Experiment 2.1: Batch Hard Miner + Contrastive Loss

In [7]:
TRAIN = False

EXPERIMENT_NAME = 'contrastive_batchhard_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2542728/2327040488.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 72.84 | 83.00  |
+----------+-------+--------+
R1: 0.7283873389215564
R5: 0.8299762292005505


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 32.50 | 48.60  |
+----------+-------+--------+
R1: 0.325
R5: 0.486


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 49.52 | 61.27  |
+----------+-------+--------+
R1: 0.49523809523809526
R5: 0.6126984126984127


#### Experiment 2.2: DistanceWeightedMiner + SupConLoss


In [8]:
TRAIN = False

EXPERIMENT_NAME = 'supcon_distanceweighted_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.DistanceWeightedMiner()
    loss_fn = losses.SupConLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2542728/4286773646.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 70.81 | 82.30  |
+----------+-------+--------+
R1: 0.7081196046540723
R5: 0.8229700988364819


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 28.80 | 45.60  |
+----------+-------+--------+
R1: 0.288
R5: 0.456


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 43.49 | 57.46  |
+----------+-------+--------+
R1: 0.43492063492063493
R5: 0.5746031746031746


#### Experiment 2.3: PairMargin + FastAP

In [9]:
TRAIN = False

EXPERIMENT_NAME = 'fastap_pairmargin_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.PairMarginMiner()
    loss_fn = losses.FastAPLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2542728/4279425740.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 71.34 | 82.46  |
+----------+-------+--------+
R1: 0.7133742024271237
R5: 0.8245965219567121


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 29.30 | 45.80  |
+----------+-------+--------+
R1: 0.293
R5: 0.458


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 40.63 | 53.97  |
+----------+-------+--------+
R1: 0.40634920634920635
R5: 0.5396825396825397


#### Experiment 2.4: No Miner + MultiSimilarityLoss

In [10]:
TRAIN = False

EXPERIMENT_NAME = 'multisimilarity_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = None
    loss_fn = losses.MultiSimilarityLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2542728/1054108450.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 70.62 | 81.87  |
+----------+-------+--------+
R1: 0.7062429625922683
R5: 0.818716376829726


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 29.10 | 47.00  |
+----------+-------+--------+
R1: 0.291
R5: 0.47


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 42.86 | 61.59  |
+----------+-------+--------+
R1: 0.42857142857142855
R5: 0.6158730158730159


# Phase 3: Optimizer, LR, WD and LR Scheduler

## SGD, 1e-2, 1e-5

In [11]:
TRAIN = False

EXPERIMENT_NAME = 'SGD_1e2_1e5'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-5,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2542728/3927305494.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 73.98 | 84.22  |
+----------+-------+--------+
R1: 0.7397723007631678
R5: 0.8422369573376705


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 37.20 | 53.90  |
+----------+-------+--------+
R1: 0.372
R5: 0.539


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 50.48 | 66.98  |
+----------+-------+--------+
R1: 0.5047619047619047
R5: 0.6698412698412698


## 3.2 ADAMW, 1e-4, 1e-4

In [12]:
TRAIN = False

EXPERIMENT_NAME = 'AdamW_1e4_1e4'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.AdamW(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-4)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2542728/3593095194.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 75.03 | 85.25  |
+----------+-------+--------+
R1: 0.7502814963092707
R5: 0.8524959339421995


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 37.50 | 53.80  |
+----------+-------+--------+
R1: 0.375
R5: 0.538


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 49.52 | 64.76  |
+----------+-------+--------+
R1: 0.49523809523809526
R5: 0.6476190476190476


## 3.3 ADAM, 1e-4, 1e-5

In [13]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2542728/1025689770.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 75.14 | 85.74  |
+----------+-------+--------+
R1: 0.7514074815463531
R5: 0.8573752033028901


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 38.40 | 54.80  |
+----------+-------+--------+
R1: 0.384
R5: 0.548


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 50.16 | 65.71  |
+----------+-------+--------+
R1: 0.5015873015873016
R5: 0.6571428571428571


# LR SCHEDULER

## Poly

In [17]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5_custom'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2542728/1329618683.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 77.46 | 87.13  |
+----------+-------+--------+
R1: 0.7745527336419367
R5: 0.8712623545602403


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 42.60 | 58.40  |
+----------+-------+--------+
R1: 0.426
R5: 0.584


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 53.02 | 70.48  |
+----------+-------+--------+
R1: 0.5301587301587302
R5: 0.7047619047619048


## Cosine

In [16]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5_cosine'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'cosine',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2542728/54693177.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 74.99 | 85.39  |
+----------+-------+--------+
R1: 0.7499061678969098
R5: 0.8538721381208557


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 39.00 | 56.80  |
+----------+-------+--------+
R1: 0.39
R5: 0.568


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 46.67 | 63.49  |
+----------+-------+--------+
R1: 0.4666666666666667
R5: 0.6349206349206349


## MIXVPR

In [2]:
TRAIN = True

EXPERIMENT_NAME = 'Adam_1e4_1e5_mixvpr_custom'

model = GetModel(aggregator='mixvpr', input_size=256, output_size=512)
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'patience': 45,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

  0%|          | 0/45 [00:00<?, ?it/s]/home/arda/anaconda3/envs/geo/lib/python3.10/site-packages/faiss/contrib/torch_utils.py:51: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  x.storage().data_ptr() + x.storage_offset() * 4)
  2%|▏         | 1/45 [04:15<3:07:03, 255.08s/it]



+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 67.50 | 77.13  |
+----------+-------+--------+
Epoch: 1
R1: 0.6749655948955335
R5: 0.7712998874014763
Loss: 0.9960341378332327
Acc: 0.0
Yeyyy! New best model!


  4%|▍         | 2/45 [08:29<3:02:39, 254.88s/it]



+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 71.27 | 80.13  |
+----------+-------+--------+
Epoch: 2
R1: 0.7127486550731891
R5: 0.8013261603903415
Loss: 0.9832604944515533
Acc: 0.0
Yeyyy! New best model!


  7%|▋         | 3/45 [12:44<2:58:28, 254.96s/it]

...



+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 74.73 | 83.62  |
+----------+-------+--------+
Epoch: 7
R1: 0.7472788690103841
R5: 0.8362317027398974
Loss: 0.9795810244144342
Acc: 0.0
Yeyyy! New best model!


 18%|█▊        | 8/45 [33:51<2:36:12, 253.30s/it]



+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 74.92 | 82.99  |
+----------+-------+--------+
Epoch: 8
R1: 0.7491555110721881
R5: 0.8298511197297636
Loss: 0.9804661216827246
Acc: 0.0


 20%|██        | 9/45 [38:05<2:32:07, 253.54s/it]



+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 75.22 | 83.52  |
+----------+-------+--------+
Epoch: 9
R1: 0.7521581383710747
R5: 0.8352308269736018
Loss: 0.9799689699095279
Acc: 0.0
Yeyyy! New best model!


In [ ]:
TRAIN = True

EXPERIMENT_NAME = 'Adam_1e4_1e5_mixvpr_1024_custom'

model = GetModel(aggregator='mixvpr', input_size=256, output_size=1024)
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'patience': 45,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

In [ ]:
TRAIN = True

EXPERIMENT_NAME = 'Adam_1e4_1e5_mixvpr_2048_custom'

model = GetModel(aggregator='mixvpr', input_size=256, output_size=2048)
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'patience': 45,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

/tmp/ipykernel_2542728/2978699910.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))


RuntimeError: Error(s) in loading state_dict for GetModel:
	Unexpected key(s) in state_dict: "model_depth.cropped_resnet.0.weight", "model_depth.cropped_resnet.1.weight", "model_depth.cropped_resnet.1.bias", "model_depth.cropped_resnet.1.running_mean", "model_depth.cropped_resnet.1.running_var", "model_depth.cropped_resnet.1.num_batches_tracked", "model_depth.cropped_resnet.4.0.conv1.weight", "model_depth.cropped_resnet.4.0.bn1.weight", "model_depth.cropped_resnet.4.0.bn1.bias", "model_depth.cropped_resnet.4.0.bn1.running_mean", "model_depth.cropped_resnet.4.0.bn1.running_var", "model_depth.cropped_resnet.4.0.bn1.num_batches_tracked", "model_depth.cropped_resnet.4.0.conv2.weight", "model_depth.cropped_resnet.4.0.bn2.weight", "model_depth.cropped_resnet.4.0.bn2.bias", "model_depth.cropped_resnet.4.0.bn2.running_mean", "model_depth.cropped_resnet.4.0.bn2.running_var", "model_depth.cropped_resnet.4.0.bn2.num_batches_tracked", "model_depth.cropped_resnet.4.1.conv1.weight", "model_depth.cro

# Phase 4: Depth

In [1]:
from models.helper import GetModel, GetMultiModel, GetDeAttNet
from dataloaders.train_loader import GSVCitiesDataset
from dataloaders.test_loader import TestDataset
from torch.utils.data import DataLoader
import torch
import os
from pytorch_metric_learning import losses, miners
from train import train_model
from evaluation import eval_model
from utils.lr_scheduler import custom_scheduler
import random
import numpy as np
import torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)


train_dataset = GSVCitiesDataset(generated_data_prob=0, multi_model=True)
test_dataset = TestDataset(multi_model=True)

num_workers = 16
train_dataloader = DataLoader(train_dataset, 
                              batch_size=100, 
                              shuffle=True, 
                              num_workers=num_workers, 
                              pin_memory=True
                              )

test_dataloader = DataLoader(test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
sf_xs_test_dataset = TestDataset(path='./data/sf_xs/test', multi_model=True)
sf_xs_test_dataloader = DataLoader(sf_xs_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
tokyo_test_dataset = TestDataset(path='./data/tokyo_xs/test', multi_model=True)
tokyo_test_dataloader = DataLoader(tokyo_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )


torch.cuda.empty_cache()

# Concat

In [21]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_depth_no_fusion_arda_512'

model = GetMultiModel(aggregator='gem', use_fusion=False, output_size=512)
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

Evaluating on test sets
--------------------


/tmp/ipykernel_2542728/405888062.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 79.76 | 88.82  |
+----------+-------+--------+
R1: 0.7975728762667333
R5: 0.8881521331164769


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 46.70 | 61.30  |
+----------+-------+--------+
R1: 0.467
R5: 0.613


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 66.67 | 82.86  |
+----------+-------+--------+
R1: 0.6666666666666666
R5: 0.8285714285714286


In [22]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_depth_no_fusion_arda_1024'

model = GetMultiModel(aggregator='gem', use_fusion=False, output_size=1024)
model = model.to('cuda')
# model.load_state_dsict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

Evaluating on test sets
--------------------


/tmp/ipykernel_2542728/3056191355.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 79.79 | 88.69  |
+----------+-------+--------+
R1: 0.7979482046790942
R5: 0.8869010384086076


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 49.30 | 64.70  |
+----------+-------+--------+
R1: 0.493
R5: 0.647


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 69.21 | 83.49  |
+----------+-------+--------+
R1: 0.692063492063492
R5: 0.834920634920635


# Depth+ mixvpr

In [5]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_depth_mixvpr_arda_512'

model = GetMultiModel(aggregator='mixvpr', use_fusion=False, input_size=512, output_size=512)
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

Evaluating on test sets
--------------------


/tmp/ipykernel_2556397/2687052716.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 82.20 | 89.69  |
+----------+-------+--------+
R1: 0.8219692230701864
R5: 0.8969097960715626


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 58.20 | 68.80  |
+----------+-------+--------+
R1: 0.582
R5: 0.688


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 76.51 | 85.40  |
+----------+-------+--------+
R1: 0.765079365079365
R5: 0.8539682539682539


In [3]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_depth_mixvpr_arda_1024'

model = GetMultiModel(aggregator='mixvpr', use_fusion=False, input_size=512, output_size=1024)
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

Evaluating on test sets
--------------------


/tmp/ipykernel_2556397/1770950872.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 83.02 | 89.95  |
+----------+-------+--------+
R1: 0.8302264481421243
R5: 0.8995370949580883


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 57.00 | 69.40  |
+----------+-------+--------+
R1: 0.57
R5: 0.694


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 75.24 | 88.25  |
+----------+-------+--------+
R1: 0.7523809523809524
R5: 0.8825396825396825


In [4]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_depth_mixvpr_arda_2048'

model = GetMultiModel(aggregator='mixvpr', use_fusion=False, input_size=512, output_size=2048)
model = model.to('cuda')
model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

Evaluating on test sets
--------------------


/tmp/ipykernel_2556397/2019231328.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
/tmp/ipykernel_25



+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 82.85 | 90.12  |
+----------+-------+--------+
R1: 0.8284749155511072
R5: 0.9011635180783185


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 58.80 | 71.60  |
+----------+-------+--------+
R1: 0.588
R5: 0.716


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 77.46 | 88.25  |
+----------+-------+--------+
R1: 0.7746031746031746
R5: 0.8825396825396825


# DEATTNET

In [25]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_deattnet'

model = GetDeAttNet(aggregator='gem')
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

Evaluating on test sets
--------------------


/tmp/ipykernel_2542728/1937420694.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 77.02 | 86.48  |
+----------+-------+--------+
R1: 0.7701739021643939
R5: 0.8647566620793194


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 40.30 | 55.50  |
+----------+-------+--------+
R1: 0.403
R5: 0.555


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 53.33 | 66.67  |
+----------+-------+--------+
R1: 0.5333333333333333
R5: 0.6666666666666666
